In [ ]:
from skimage import io
from skimage import exposure

import os
import gzip
import gc
import pprint as pp
import numpy as np
import pickle
import matplotlib.pyplot as plt
import logging
import random
from datetime import datetime

import keras
from keras.models import load_model
from keras.utils import to_categorical

In [ ]:
log_location = "../../logs/"
PATH_TO_INTERMEDIATE = "../../data/segmentation.pickle"
PATH_TO_MODELS = "../../data/segmentation.models"

In [ ]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'validation.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

In [ ]:
kernel_size = 48
half_kernel = int(kernel_size/2)
num_classes = 2

In [ ]:
validation_data = []
for current_model in os.listdir(PATH_TO_INTERMEDIATE):
    if not (current_model.endswith('x.pickle') and current_model.startswith('validation')):
        continue
    validation_data.append(os.path.join(PATH_TO_INTERMEDIATE,current_model))  
    

In [ ]:
validation_data

In [ ]:
def log(msg):
    logging.debug(msg)
 
def print_log(msg):
    log(msg)
    print(msg)   

In [ ]:
model_config = {}
model_evaluation_file_name = os.path.join(PATH_TO_MODELS,'model.evaluation.pickle')
model_evaluation = {}

for current_model in os.listdir(PATH_TO_MODELS):
    if not (current_model.endswith(".pickle") and current_model.startswith("2")):
        continue
    current_model = os.path.join(PATH_TO_MODELS,current_model)
    with gzip.open(current_model,'rb') as f:
            model_config = pickle.load(f)  

    model = load_model(model_config['model_file_name'])
    for current_data_set in validation_data:
        with gzip.open(current_data_set,'rb') as f:
            validation_x = pickle.load(f)
        with gzip.open(current_data_set.replace('.x.pickle','.y.pickle'),'rb') as f:
            validation_y = pickle.load(f)

        validation_x = (validation_x / 128) - 1
        validation_x = validation_x.reshape(validation_x.shape[0], kernel_size, kernel_size,1)
        validation_y = validation_y.reshape(validation_y.shape[0])
        validation_y = keras.utils.to_categorical(validation_y, num_classes)
            
        current_data_set_name = current_data_set.split('/')[-1]
        model_name = model_config['model_file_name'].split('/')[-1]
        print_log(model_name)
        print_log(current_data_set_name)
        model_evaluation[model_name]= {}
        model_evaluation[model_name]['name']=model_name
        model_evaluation[model_name]['scores'] = {}
        
        score = model.evaluate(validation_x, validation_y, verbose=1)
        model_evaluation[model_name]['scores'][current_data_set_name] = score
        print_log(score)
        print_log(model_evaluation)
        with gzip.open(model_evaluation_file_name,'wb') as f:
            pickle.dump(model_evaluation, f, protocol=pickle.HIGHEST_PROTOCOL)  
        